#how to setup rtmp server, please reference below url
https://www.cnblogs.com/nowgood/p/ffmpegnginx.html

#send rtmp data from pi4 to server, can use follow command
ffmpeg -f v4l2 -r 24 -video_size vga -pix_fmt yuv420p12be -i /dev/video0 -b:v 500k -c:v h264_omx -preset ultrafast -an -f flv rtmp://52.80.130.52/live/123456


In [ ]:
!pip install gluoncv -i https://opentuna.cn/pypi/web/simple

In [ ]:
!pip install bokeh==1.3.1 -i https://opentuna.cn/pypi/web/simple

In [1]:
import boto3
from IPython.display import clear_output, Image, display, HTML
import numpy as np
import cv2
import base64
from bokeh.plotting import figure
from bokeh.io import output_notebook, show, push_notebook
import time
output_notebook()

Loading BokehJS ...

In [2]:
# import sagemaker
# sagemaker_endpoint_name = 'object-detection-2019-10-18-17-56-41-925'
# sagemaker_endpoint = sagemaker.predictor.RealTimePredictor(sagemaker_endpoint_name)
# sagemaker_endpoint.content_type = 'image/jpeg'
from image_classification import ImageClassification
imageClassification = ImageClassification()

for i in range(10):
    # use GluonCV
    detect_start = time.time()
    frame = cv2.imread('../images/1.jpg')
    scores = imageClassification.classify_image(frame)
    #print('scores:', scores)
    detect_end = time.time()
    print('detect time:', detect_end-detect_start)

[2021-11-08 08:07:31.471 ip-172-16-5-225:2192 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2021-11-08 08:07:31.495 ip-172-16-5-225:2192 INFO profiler_config_parser.py:111] Unable to find config at /opt/ml/input/config/profilerconfig.json. Profiler is disabled.
detect time: 0.7973179817199707
detect time: 0.05270504951477051
detect time: 0.05488014221191406
detect time: 0.05238699913024902
detect time: 0.051278114318847656
detect time: 0.052320241928100586
detect time: 0.05222797393798828
detect time: 0.0504307746887207
detect time: 0.09525752067565918
detect time: 0.05455493927001953


In [3]:
class VideoPlayer(object):
    def __init__(self):
        self._init = False
        self._myImage = None
        
    def __call__(self, frame):
        if frame is None:
            return
        if self._init is False:
            self.init_display(frame)
            self._init = True
        else:
            self.update_display(frame)

    def init_display(self, frame):
        assert frame is not None
        frame=cv2.cvtColor(frame, cv2.COLOR_BGR2RGBA) # because Bokeh expects a RGBA image
#         frame=cv2.flip(frame, -1) # because Bokeh flips vertically
        frame=cv2.flip(frame, 0) # because Bokeh flips vertically
        width=frame.shape[1]
        height=frame.shape[0]
        p = figure(x_range=(0,width), y_range=(0,height), output_backend="webgl", width=width, height=height)
        self._myImage = p.image_rgba(image=[frame], x=0, y=0, dw=width, dh=height)
        show(p, notebook_handle=True)
    
    def update_display(self, frame):
        assert frame is not None
        frame=cv2.cvtColor(frame, cv2.COLOR_BGR2RGBA)
#         frame=cv2.flip(frame, -1)
        frame=cv2.flip(frame, 0) 
        self._myImage.data_source.data['image']=[frame]
        push_notebook()

In [4]:
def detection_result_process(frame, classes, scores, pre_msg, pre_msg2):
    thres = 0.5
    cv2.putText(frame, str(classes[np.argmax(scores[0])]), (100, 100), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=2, thickness=1, color=(0, 0, 255))
    msg = pre_msg
    msg2 = pre_msg2
    return frame, msg, msg2

In [5]:
"""
need add rtmp support for sagemaker, please follow this page
https://answers.opencv.org/question/180776/build-opencv-with-ffmpeg-support/
"""
url="rtmp://52.81.133.221/live/123456"

vcap = cv2.VideoCapture(url)
player = VideoPlayer()

pre_msg = ''
pre_msg2 = ''
run_mode = 2  # 1 for original image, 2 for image classification
fps = 8  # 24

print("Video start")
while(True):
    # Capture frame-by-frame
    read_start = time.time()
    if run_mode == 1:
        for i in range(fps):
            ret, frame = vcap.read()
    elif run_mode == 2:
        ret, frame = vcap.read()
    read_end = time.time()
#     print('read time:', read_end-read_start)

    if frame is not None:
        start = time.time()
        frame = cv2.flip(frame, -1)
        
        # save image
        if run_mode == 1:
            filename = '../images/echo/frame_'+str(time.time())+'.jpg'
            cv2.imwrite(filename, frame)
            
            player(frame)

            print(filename)

        elif run_mode == 2:
            frame = frame[0:280, 180:460]
            
            # use SageMaker
            #with open(filename, 'rb') as image:
            #    f = image.read()
            #    b = bytearray(f)
            #result = sagemaker_endpoint.predict(b)
            #print(json.loads(result))

            # use GluonCV
            detect_start = time.time()
            scores = imageClassification.classify_image(frame)
            #print('scores:', scores)
            detect_end = time.time()
#             print('detect time:', detect_end-detect_start)

            frame, msg, msg2 = detection_result_process(frame, imageClassification.classes, scores, pre_msg, pre_msg2)
            #print(time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(detect_end)), 'msg:', msg)
            #print(time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(detect_end)), 'msg2:', msg2)
            pre_msg = msg
            pre_msg2 = msg2

            # Display the resulting frame
            #cv2.imwrite(filename, frame)
            player(frame)
            end = time.time()
            #print('all time:', end-start)

            #t = 1
            #time.sleep(t)
            #vcap.set(cv2.CAP_PROP_POS_FRAMES, t*25)

            #t = 0.2
            #sleep_time = t-(end-start)
            #if sleep_time >= 0:
            #    time.sleep(sleep_time)
            #    vcap.set(cv2.CAP_PROP_POS_FRAMES, sleep_time*25)

            #skip_frame = max(0, 25-int(1/(end-start)))
            #print('skip_frame:', skip_frame)
            #if skip_frame > 0:
            #    vcap.set(cv2.CAP_PROP_POS_FRAMES, skip_frame)

        if run_mode == 1:
            t = 1
            time.sleep(t)
#         elif run_mode == 2:
#             t = 1/fps
#             time.sleep(t)
            
        #vcap.set(cv2.CAP_PROP_POS_FRAMES, t*25)
        #break

        # Press q to close the video windows before it ends if you want
        #if cv2.waitKey(22) & 0xFF == ord('q'):
        #    break
    else:
        print("Frame is None")
        break

# When everything done, release the capture
vcap.release()
print("Video stop")

Video start


KeyboardInterrupt: 